In [1]:
require(tidyverse)
require(lubridate)
require(magrittr)
require(text2vec)
require(tokenizers)
require(stopwords)
require(xgboost)
require(Matrix)
require(stringr)
require(stringi)
require(forcats)
require(dplyr)
library(ade4)
library(data.table)
library(pryr)
set.seed(0)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: text2vec
Loading required package: tokenizers
Loading required package: stopwords
Loading required package: xgboost

Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice

Loading 

In [2]:
?read_csv

In [57]:
cat("Loading data...\n")
num = 1000
tr <- read_csv("../input/att_train.csv", n_max = num) 
#te <- read_csv("../input/att_test.csv", n_max = num)
tri <- 1:nrow(tr)
rm(tr);gc()

tr_te <- read_csv("/media/extend/cache/total_0_1503424.csv", n_max = num * 2) %>%
    select(-X1, -deal_probability)
y <- read_csv("/media/extend/cache/total_y_0_1503424.csv", n_max = num)$deal_probability

Loading data...


Parsed with column specification:
cols(
  .default = col_double(),
  item_id = col_character(),
  user_id = col_character(),
  region = col_character(),
  city = col_character(),
  parent_category_name = col_character(),
  category_name = col_character(),
  param_1 = col_character(),
  param_2 = col_character(),
  param_3 = col_character(),
  title = col_character(),
  description = col_character(),
  item_seq_number = col_integer(),
  activation_date = col_date(format = ""),
  user_type = col_character(),
  image = col_character(),
  reg_Time_zone = col_integer()
)
See spec(...) for full column specifications.


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1511129,80.8,2637877,140.9,2637877,140.9
Vcells,2379382,18.2,4701432,35.9,4701094,35.9


Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_integer(),
  item_id = col_character(),
  category_name = col_integer(),
  city = col_integer(),
  deal_probability = col_character(),
  description = col_character(),
  image_top_1 = col_integer(),
  item_seq_number = col_integer(),
  param_1 = col_integer(),
  param_2 = col_integer(),
  param_3 = col_integer(),
  parent_category_name = col_integer(),
  region = col_integer(),
  title = col_character(),
  user_id = col_integer(),
  user_type = col_integer(),
  reg_Time_zone = col_integer(),
  n_user_items = col_integer(),
  Weekday = col_integer(),
  img_label = col_character()
  # ... with 24 more columns
)
See spec(...) for full column specifications.
Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_integer(),
  item_id = col_character(),
  deal_probability = col_double()
)


In [58]:
catcols = c('user_id', 'city', 'region', 
            'parent_category_name', 'category_name',
            'param_1','param_2','param_3',
            'user_type', 'Weekday','img_label')
for (col in catcols){
    tr_te[[col]] <- ifelse(is.na(tr_te[[col]]), 'none', tr_te[[col]])
}

In [59]:
tr_te <- tr_te %>% 
  mutate(no_img = is.na(image) %>% as.integer(),
         no_dsc = is.na(description) %>% as.integer(),
         no_p1 = is.na(param_1) %>% as.integer(), 
         no_p2 = is.na(param_2) %>% as.integer(), 
         no_p3 = is.na(param_3) %>% as.integer(),
         titl_capE = str_count(title, "[A-Z]"),
         titl_capR = str_count(title, "[А-Я]"),
         desc_capE = str_count(description, "[A-Z]"),
         desc_capR = str_count(description, "[А-Я]"),
         user_type = factor(user_type),
         category_name = factor(category_name),
         parent_category_name = factor(parent_category_name), 
         region = factor(region),
         img_label = factor(img_label),
         param_1 = factor(param_1),
         param_2 = factor(param_2),
         param_3 = factor(param_3) %>% fct_lump(prop = 0.00005),
         city =  factor(city) %>% fct_lump(prop = 0.0003),
         user_id = factor(user_id) %>% fct_lump(prop = 0.000025),
         Weekday = factor(Weekday),
         n_user_items = log1p(n_user_items),
         avg_days_up_user = log1p(avg_days_up_user),
         avg_times_up_user = log1p(avg_times_up_user),
         txt = paste(title, description, sep = " "))%>% 
  select(-item_id, -title, -description) %>% 
  replace_na(list(image_top_1 = -1, price = -1, 
                  param_1   = -1, param_2 = -1, param_3 = -1, 
                  titl_len  = 0,  desc_len  = 0,
                  titl_capE = 0,  titl_capR = 0, 
                  titl_lowE = 0,  titl_lowR = 0, 
                  desc_cap  = 0,
                  titl_pun  = 0,  desc_pun = 0,
                  titl_dig  = 0,  desc_dig = 0,
                  desc_capE = 0,
                  desc_capR = 0)) %T>% 
  glimpse()

Warning message in is.na(image):
“is.na() applied to non-(list or vector) of type 'closure'”Warning message in `[<-.factor`(`*tmp*`, !is_complete(data[[var]]), value = -1):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, !is_complete(data[[var]]), value = -1):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, !is_complete(data[[var]]), value = -1):
“invalid factor level, NA generated”

Observations: 2,000
Variables: 102
$ category_name                                             <fct> 42, 22, 2...
$ city                                                      <fct> 462, 1314...
$ image_top_1                                               <dbl> 13, 2723,...
$ item_seq_number                                           <int> 2, 19, 9,...
$ param_1                                                   <fct> 249, 122,...
$ param_2                                                   <fct> 112, 112,...
$ param_3                                                   <fct> 1217, 121...
$ parent_category_name                                      <fct> 4, 2, 0, ...
$ price                                                     <dbl> 5.992000,...
$ region                                                    <fct> 19, 16, 1...
$ user_id                                                   <fct> 884270, 2...
$ user_type                                                 <fct> 1, 1, 1, ...
$ latitude       

In [60]:
for (col in colnames(tr_te)){
    cnt_zero = sum(is.na(tr_te[col]))
    if(cnt_zero > 0){
        #cat(col, cnt_zero, "\n")
        tr_te[[col]] <- ifelse(is.na(tr_te[[col]]), 0, tr_te[[col]])
    }  
}
    #sum(is.na(tr_te))

In [61]:
n <- nrow(tr_te)
nlevels <- sapply(tr_te[catcols], nlevels)
i <- rep(seq_len(n), ncol(tr_te[catcols]))
j <- unlist(lapply(tr_te[catcols], as.integer)) +
     rep(cumsum(c(0, head(nlevels, -1))), each = n)
x <- 1
sparse_cats <- sparseMatrix(i = i, j = j, x = x)

In [62]:
object_size(sparse_cats)

278 kB

In [63]:
spdf <- tr_te %>% 
    sparse.model.matrix( ~ . - 1, .)

In [ ]:
rm(tr, te); gc()

In [64]:
#---------------------------
cat("Parsing text...\n")
it <- tr_te %$%
  str_to_lower(txt) %>%
  str_replace_all("[^[:alpha:]]", " ") %>%
  str_replace_all("\\s+", " ") %>%
  tokenize_word_stems(language = "russian") %>% 
  itoken()

vect <- create_vocabulary(it, ngram = c(1, 1), stopwords = stopwords("ru")) %>%
  prune_vocabulary(term_count_min = 3, doc_proportion_max = 0.4, vocab_term_max = 12500) %>% 
  vocab_vectorizer()

m_tfidf <- TfIdf$new(norm = "l2", sublinear_tf = T)
tfidf <-  create_dtm(it, vect) %>% 
  fit_transform(m_tfidf)

rm(it, vect, m_tfidf); gc()

Parsing text...


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1523591,81.4,2637877,140.9,2637877,140.9
Vcells,2721196,20.8,5721718,43.7,5694450,43.5


In [66]:

cat("Preparing data...\n")
X <- tr_te %>% 
  select(-txt) %>% 
  sparse.model.matrix(~ . - 1, .) %>% 
  cbind(tfidf)
dim(X)

dtest <- xgb.DMatrix(data = X[-tri, ])
X <- X[tri, ]; gc()
cols <- colnames(X)

Preparing data...


[1] 2000 6132

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1511952,80.8,2637877,140.9,2637877,140.9
Vcells,2360823,18.1,5721718,43.7,5721654,43.7


In [67]:
idx = seq(1, length(y))
idx = sample(idx)
fold = 5
folds <- cut(idx,breaks=fold,labels=FALSE)

In [68]:
p <- list(objective = "reg:logistic",
          booster = "gbtree",
          eval_metric = "rmse",
          nthread = 6,
          eta = 0.05,
          max_depth = 18,
          min_child_weight = 11,
          gamma = 0,
          subsample = 0.85,
          colsample_bytree = 0.7,
          alpha = 2.0,
          lambda = 0,
          nrounds = 2000)

i = 1
#Segement your data by fold using the which() function 
testIndexes <- which(folds==i,arr.ind=TRUE)

testX <- X[testIndexes, ]
testY <- y[testIndexes]

trainX <- X[-testIndexes, ]
trainY <- y[-testIndexes]
#Use the test and train data partitions however you desire...

dtrain <- xgb.DMatrix(data = trainX, label = trainY)
dval <- xgb.DMatrix(data = testX, label = testY)
m_xgb <- xgb.train(p, dtrain, p$nrounds, list(train = dtrain ,val = dval), print_every_n = 10, early_stopping_rounds = 10)


[1]	train-rmse:0.427887	val-rmse:0.422311 
Multiple eval metrics are present. Will use val_rmse for early stopping.
Will train until val_rmse hasn't improved in 10 rounds.

[11]	train-rmse:0.324065	val-rmse:0.325286 
[21]	train-rmse:0.273450	val-rmse:0.282526 
[31]	train-rmse:0.249698	val-rmse:0.266069 
[41]	train-rmse:0.237859	val-rmse:0.260953 
[51]	train-rmse:0.229359	val-rmse:0.258153 
[61]	train-rmse:0.223122	val-rmse:0.256652 
[71]	train-rmse:0.218990	val-rmse:0.256340 
[81]	train-rmse:0.214875	val-rmse:0.256491 
Stopping. Best iteration:
[72]	train-rmse:0.218315	val-rmse:0.256298



In [ ]:
?xgb.train

In [1]:
catcols = c('user_id', 'city', 'region', 
            'parent_category_name', 'category_name',
            'param_1','param_2','param_3',
            'user_type')

In [2]:
c(catcols, 'tmp')

[1] "user_id"              "city"                 "region"              
 [4] "parent_category_name" "category_name"        "param_1"             
 [7] "param_2"              "param_3"              "user_type"           
[10] "tmp"

In [ ]:
?read_csv